<img align="right" width="400" src="https://www.fhnw.ch/de/++theme++web16theme/assets/media/img/fachhochschule-nordwestschweiz-fhnw-logo.svg" alt="FHNW Logo">


# Wrong Label Identification

by Fabian Märki

## Summary
The aim of this notebook is check if it is possible to automatically identify wrong labeled text data.

There is always an uncertainty about the quality of training data. This quality is one of the key factors that influence the prediction capabilities of a model. This notebook is an attempt to show how wrong labels can be identified.   

## Sources
- [cleanlab](https://pypi.org/project/cleanlab)
- [Cleanlab: A Python Package for ML and Deep Learning on Datasets with Label Errors](https://l7.curtisnorthcutt.com/cleanlab-python-package)
- [Find Label Issues with Confident Learning for NLP](https://www.depends-on-the-definition.com/confident-learning-for-nlp)
- [An Introduction to Confident Learning: Finding and Learning with Label Errors in Datasets](https://l7.curtisnorthcutt.com/confident-learning)

This notebook does not contain assigments: <font color='red'>Enjoy.</font>

<a href="https://colab.research.google.com/github/markif/2024_FS_CAS_NLP_LAB_Notebooks/blob/master/05_a_Wrong_Label_Identification.ipynb">
  <img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
%%capture

!pip install 'fhnw-nlp-utils>=0.8.0,<0.9.0'

from fhnw.nlp.utils.storage import download
from fhnw.nlp.utils.storage import load_dataframe


import pandas as pd
import numpy as np

In [2]:
from fhnw.nlp.utils.system import set_log_level
from fhnw.nlp.utils.system import system_info

set_log_level()
print(system_info())

OS name: posix
Platform name: Linux
Platform release: 5.19.0-41-generic
Python version: 3.8.10
CPU cores: 6
RAM: 31.12GB total and 20.03GB available
Tensorflow version: 2.12.0
GPU is available
GPU is a NVIDIA GeForce RTX 2070 with Max-Q Design with 8192MiB


In [3]:
%%time
download("https://drive.switch.ch/index.php/s/0hE8wO4FbfGIJld/download", "data/german_doctor_reviews_tokenized.parq")
data = load_dataframe("data/german_doctor_reviews_tokenized.parq")
data.shape

CPU times: user 7.05 s, sys: 1.5 s, total: 8.55 s
Wall time: 5.04 s


(350087, 10)

In [4]:
data.head(3)

,text_original,rating,text,label,sentiment,token_clean,text_clean,token_lemma,token_stem,token_clean_stopwords
0,Ich bin franzose und bin seit ein paar Wochen ...,2.0,Ich bin franzose und bin seit ein paar Wochen ...,positive,1,"[ich, bin, franzose, und, bin, seit, ein, paar...",ich bin franzose und bin seit ein paar wochen ...,"[franzose, seit, paar, woche, muenchen, zahn, ...","[franzos, seit, paar, woch, muench, ., zahn, s...","[franzose, seit, paar, wochen, muenchen, ., za..."
1,Dieser Arzt ist das unmöglichste was mir in me...,6.0,Dieser Arzt ist das unmöglichste was mir in me...,negative,-1,"[dieser, arzt, ist, das, unmöglichste, was, mi...",dieser arzt ist das unmöglichste was mir in me...,"[arzt, unmöglich, leben, je, begegnen, unfreun...","[arzt, unmog, leb, je, begegnet, unfreund, ,, ...","[arzt, unmöglichste, leben, je, begegnet, unfr..."
2,Hatte akute Beschwerden am Rücken. Herr Magura...,1.0,Hatte akute Beschwerden am Rücken. Herr Magura...,positive,1,"[hatte, akute, beschwerden, am, rücken, ., her...",hatte akute beschwerden am rücken . herr magur...,"[akut, beschwerde, rücken, magura, erster, arz...","[akut, beschwerd, ruck, ., magura, erst, arzt,...","[akute, beschwerden, rücken, ., magura, erste,..."


In [5]:
# remove all neutral sentimens
data = data.loc[(data["label"] != "neutral")]
data.shape

(331187, 10)

In [6]:
# set rating to wrong values
data.at[1,"sentiment"] = 1
data.at[1,"label"] = "positive"
data.at[19,"sentiment"] = 1
data.at[19,"label"] = "positive"
# data.at[357895,"rating"] = -1
# data.at[357895,"sentiment"] = "negative"
# data.at[357896,"rating"] = -1
# data.at[357896,"sentiment"] = "negative"

data["wrong_label"] = 0
data.at[1,"wrong_label"] = 1
data.at[19,"wrong_label"] = 1
#data.at[357895,"wrong_label"] = 1
#data.at[357896,"wrong_label"] = 1

This time we use all data for training.

In [7]:
X_train, y_train = data["token_lemma"], data["label"]

### Base Classifier

We need a trained classifier. Let's use our base classifier...

In [8]:
%%time

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.calibration import CalibratedClassifierCV

pipe = Pipeline([
         ("vec", CountVectorizer(ngram_range=(1, 2), tokenizer=lambda x: x,preprocessor=lambda x: x, stop_words=None)),
         ('tfidf', TfidfTransformer()),
         ("clf", CalibratedClassifierCV(estimator=SGDClassifier(alpha=5.3e-06), cv=5, method='isotonic'))
        ])

CPU times: user 227 ms, sys: 27.9 ms, total: 255 ms
Wall time: 258 ms


In [9]:
best_params = {
    #"clf__alpha": 5.3e-06, 
    "tfidf__norm": "l2", 
    "tfidf__sublinear_tf": True, 
    "tfidf__use_idf": True, 
    "vec__max_df": 0.5, 
    "vec__min_df": 0.0001,
}

In [10]:
%%time

pipe.set_params(**best_params)

pipe.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/sklearn/calibration.py:321: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


CPU times: user 26.5 s, sys: 4.88 s, total: 31.4 s
Wall time: 26.5 s


Pipeline(steps=[('vec',
                 CountVectorizer(max_df=0.5, min_df=0.0001, ngram_range=(1, 2),
                                 preprocessor=<function <lambda> at 0x7f9fbf5053a0>,
                                 tokenizer=<function <lambda> at 0x7f9fc5dd19d0>)),
                ('tfidf', TfidfTransformer(sublinear_tf=True)),
                ('clf',
                 CalibratedClassifierCV(base_estimator=SGDClassifier(alpha=5.3e-06),
                                        cv=5, method='isotonic'))])

Double check performance...

In [11]:
%%time

y_train_pred = pipe.predict(X_train)
y_train_pred_proba = pipe.predict_proba(X_train)

CPU times: user 33.7 s, sys: 225 ms, total: 33.9 s
Wall time: 33.9 s


In [12]:
from sklearn.metrics import classification_report

report = classification_report(y_train, y_train_pred)
print(report)

              precision    recall  f1-score   support

    negative       0.93      0.92      0.92     33020
    positive       0.99      0.99      0.99    298167

    accuracy                           0.98    331187
   macro avg       0.96      0.95      0.96    331187
weighted avg       0.98      0.98      0.98    331187



In [13]:
print(y_train_pred_proba[1], y_train_pred[1], data.iloc[1])
print(y_train_pred_proba[19], y_train_pred[19], data.iloc[19])

[0.98395601 0.01604399] negative text_original            Dieser Arzt ist das unmöglichste was mir in me...
rating                                                                 6.0
text                     Dieser Arzt ist das unmöglichste was mir in me...
label                                                             positive
sentiment                                                                1
token_clean              [dieser, arzt, ist, das, unmöglichste, was, mi...
text_clean               dieser arzt ist das unmöglichste was mir in me...
token_lemma              [arzt, unmöglich, leben, je, begegnen, unfreun...
token_stem               [arzt, unmog, leb, je, begegnet, unfreund, ,, ...
token_clean_stopwords    [arzt, unmöglichste, leben, je, begegnet, unfr...
wrong_label                                                              1
Name: 1, dtype: object
[0.98929444 0.01070556] negative text_original            Eine sehr unfreundliche Ärztin, so etwas habe ...
rating     

In [14]:
%%capture

!pip install cleanlab

In [15]:
from sklearn.preprocessing import LabelEncoder

lb = LabelEncoder()
y = lb.fit_transform(data["label"])

In [16]:
from cleanlab.count import estimate_py_noise_matrices_and_cv_pred_proba
from cleanlab.filter import find_label_issues

predicted_label_errors = find_label_issues(
    labels=y,
    pred_probs=y_train_pred_proba,
    #filter_by='normalized_margin', # Orders label errors
)
predicted_label_error_indices = np.argwhere(predicted_label_errors==True)

In [17]:
print("{} label errors were predicted".format(len(predicted_label_error_indices)))

1056 label errors were predicted


In [18]:
pd.options.display.max_colwidth = 500
outlier = data[predicted_label_errors]
outlier[["text_clean", "rating", "label", "sentiment"]]

,text_clean,rating,label,sentiment
1,"dieser arzt ist das unmöglichste was mir in meinem leben je begegnet ist er ist unfreundlich , sehr herablassend und medizinisch unkompetent nach seiner diagnose bin ich zu einem anderen hautarzt gegangen der mich ordentlich behandelt hat und mir auch half meine beschweerden hatten einen völlig anderen grund . nach seiner behandlung und diagnose , waren seine letzten worte ..... und tschüss alles inerhalb von ca minuten .",6.0,positive,1
19,"eine sehr unfreundliche ärztin , so etwas habe ich noch nicht erlebt , es sei denn das geld stimmt . sie kann garnicht mit menschen umgehen , dann verstehe ich echt nicht wie so ein mensch sich so einen beruf aussucht , fragen beantwortet sie sehr knapp . sehr arrogant ! ! !",6.0,positive,1
315,"nimmt sich keine zeit , man hat fast keine möglichkeit seine beschwerden zu äussern",5.0,negative,-1
766,"ich litt jahre an einem rätselhaften juckreiz , der mich dazu zwang mich mehrfach täglich an einer bestimmten stelle blutig zu kratzen . im laufe dieser jahre habe ich etliche kassendermatologen im kreis heinsberg aufgesucht . einer war unfähig , der andere hatte komplettes desinteresse , der dritte wiederum tappte schon einmal in die richtige richtung alle diese ärzte hatten die gleichen informationen bekommen wie dr. roesener . keiner der ärzte hat mir auch nur ansatzweise linderung versch...",1.0,positive,1
1157,"das empfangsteam ist wirklich derart unfreundlich und schnippisch dass ich trotz langjähriger behandlung wo ich auch zufrieden war keinen fuß mehr in die praxis setzen werde . war heute früh da , erstmal draußen warten müssen weil sie auf das klingeln erst um viertel nach reagieren obwohl im internet und an der tür uhr steht . darauf hingewiesen super arrogant und unverschämt , nicht mal antwort aufs direkt ausgesprochene tschüss bekommen . muss ich mir nicht bieten lassen .",2.0,positive,1
...,...,...,...,...
356302,"leider konnte ich diesen termin nicht wahr nehmen , und da nur ein ab da war konnte man nicht absagen",2.0,positive,1
356345,"allgemein kennt ja jeder die behauptung , ärzte mögen einen gerne veräppeln wollen , um davon finanziell zu profitieren . frau dr. herschel ist meiner erfahrung nach das gegenteil . nachdem ich leichte probleme mit dem lesen hatte , hätten mir gerne optiker eine brille verkauft . als ich dann bei frau dr. herschel saß , hat sie die tests nicht in die länge gezogen , sondern hat gut und schnell alles ermittelt , was es zu ermitteln gab . anstatt mich zum optiker zu schicken , hat sie mir wege...",1.0,positive,1
356792,"unglaublich ! ich hatte die schlechten rezensionen auf jameda gelesen , aber dachte mir ich mache mir mein eigenes bild . völlig abgehoben und nimmt einen nicht ernst . hauptsache pille verschreiben wollen , auf die antwort das ich keine pille möchte , kam die aussage das ich dann doch zu einem psychiater gehen soll .",1.0,positive,1
356885,"was wünscht man sich bei einem arztbesuch ? sicher die möglichkeit , das anliegen beschreiben zu können . vielleicht etwas empathie . was nicht ? arrogant belehrt zu werden . kein wort dazwischen zu kriegen . eine riesige liste von abrechenbaren leistungen aufgetischt zu bekommen , die in keinerlei beziehung zum anliegen stehen . dieses unwürdige schauspiel wird aufgeführt vor dem hintergrund einer aufklärungsfeindlichen und unhaltbaren quacksalberei hinsichtlich akupunktur und anderen prakt...",1.0,positive,1


**The algorithm successfully identified 1 and 19 (the ones deliberately set to the wrong sentiment).** 

It also seems that there are a few more examples (e.g. 355057, 356792, 356885 and more) where the rating seems to be wrong (which seems to be unintuitive since usually a high number (more stars) refers to a good review whereas a low number (less stars) refers to a bad review (whereas here >=5 means bad and <=2 means good). Consequently, there still seems to be room for further improvements...

In [19]:
outlier.loc[355057, ["text_clean", "rating", "label", "sentiment"]]

text_clean    seit jahren die hausärztin von meiner frau und mir . kompetent , freundlich und den anliegen der patienten immer zugewandt . wir möchten sie nicht mehr missen . auch die arzthelferin ist sehr freundlich und hilfsbereit .
rating                                                                                                                                                                                                                                 6.0
label                                                                                                                                                                                                                             negative
sentiment                                                                                                                                                                                                                               -1
Name: 355057, dtype: object